# Algorytmy szacujące wyniki zapytań COUNT DISTINCT

W referacie wykonam krótki przegląd algorytmów rozwiązywania problemu liczenia unikalnych elementów złączalnych multizbiorów (COUNT DISTINCT). Opowiem o jednym z pierwszych algorytmów probabilistycznych związanych z problemem - algorytmie Flajoleta-Martina [1] i wyjaśnię wady oryginalnej wersji algorytmu. Oprócz tego przedstawię probabilistyczną metodę odzyskiwania liczności zbioru unikalnych elementów. Następnie opowiem o modyfikacjach poprawiających dokładność szacowania liczby unikalnych elementów i wprowadzę algorytm HyperLogLog [2]. Na końcu pokażę przykład dostępnej implementacji - PostgresHLL.
#  
#  
#### Bibliografia

[1] Flajolet, Philippe; Martin, G. Nigel (1985). "Probabilistic counting algorithms for data base applications".Journal of Computer and System Sciences. 31 (2): 182–209. doi:10.1016/0022-0000(85)90041-8

[2] Flajolet, Philippe; Fusy, Éric; Gandouet, Olivier; Meunier, Frédéric (2007). "Hyperloglog: The analysis of a near-optimal cardinality estimation algorithm". Discrete Mathematics and Theoretical Computer Science proceedings. Nancy, France. AH: 127–146.

#  
#  
## Sformułowanie problemu

**Wejście:** $M$ - multizbiór

**Wyjście:** $n$ - liczba unikalnych elementów multizbioru $M$

#  
#  
## Naiwne rozwiązanie

* Inicjalizujemy pusty zbiór $U$
* Przechodzimy po wszystkich elementach multizbioru $M$ i dodajemy je do zbioru $U$
* Zwracamy liczność zbioru $U$

#### Kilka słów o złożoności
#  
Złożoności powyższego algorytmu wyglądają następująco:

* obliczeniowa - $O(|M|)$
* pamięciowa - $O(n)$

Przez naturę problemu jesteśmy zmuszeni obejrzeć każdy element multizbioru $M$ w celu ustalenia czy dany element pojawił się już wcześniej. Stąd wiemy,że złożoność obliczeniowa jaką uzyskaliśmy jest optymalna. Wadą tego rozwiązania jest natomiast złożoność pamięciowa, która jest zależna od liczby unikalnych elementów multizbioru $M$. W przypadku gdy ta liczba jest bardzo duża, powyższy algorytm staje się niewydajny. Rozważmy zatem następującą alternatywę.

## Algorytm Flajoleta-Martina

$U:=$ zbiór unikalnych wartości multizbioru $M$

Wybierzmy funkcję hashującą $hash$ taką, że $hash(U)$ jest zmienną losową o rozkładzie jednostajnym dyskretnym o wartościach w zbiorze $\{0,1,\dots,2^L-1\}$, przy czym możemy przyjąć $L=log_2|M|+10$. Wartości funkcji $hash$ możemy interpretować jako ciąg binarny długości $L$ reprezentujący odpowiadającą mu liczbę. (Przykładowo jeżeli $L = 8$, to ciąg binarny $\langle 00001000 \rangle_2$ odpowiada liczbie $8$)


Niech $x \in M$. Wtedy:

$$y = hash(x)$$

$bit(y,k)$ - $k$-ty bit reprezentacji binarnej $y$.

$$y = \sum_{k\geq0}bit(y,k)2^k$$

$\rho(y)$ - pozycja najmniej znaczącego niezerowego bitu reprezentacji binarnej $y$.

**Przykłady:**
* $\rho(5)=0$, ponieważ $5=\langle 0101 \rangle_2$
* $\rho(6)=1$, ponieważ $6=\langle 0110 \rangle_2$
* $\rho(8)=3$, ponieważ $8=\langle 1000 \rangle_2$

**Uwaga:**
Ponieważ $\rho(0)$ nie ma według powyższej definicji określonej wartości, ustalmy $\rho(0)=0$

Dla przejrzystosci obliczeń oznaczmy $H:=hash(U)$ przy czym pamiętamy, że $H$ jest zmienną losową o rozkładzie jednostajnym dyskretnym. Stąd wynika, że 

$$P(H=\langle\dots10^k\rangle_2)=2^{-k-1}$$

Powyższy napis oznacza prawdopodobieństwo, że ciąg binarny reprezentujący wartość zmiennej losowej $H$ ma na końcu $k$ zer i jedynkę na $L-k-1$ miejscu. 

Przejdźmy zatem do pierwszej częsci algorytmu:

Po przejściu przez multizbiór $M$ otrzymujemy tablicę $bitmap$ długości $L$ o wartościach $0$ lub $1$.

Zastanówmy się nad wyrażeniem
$$P(\rho(H) = k)$$
Jest to prawdopodobieństwo odwołania się do k-tej komórki tablicy. Okazuje się, że jest to nic innego jak zdefiniowane przez nas wcześniej
$$P(H=\langle\dots10^k\rangle_2)$$
Zatem wiemy, że w przybliżeniu $\frac{n}{2}$ elementów $U$ odwoływało się do $bitmap[0]$, $\frac{n}{4}$ elementów $U$ odwoływało się do $bitmap[1]$ itd. Stąd możemy wyciągnąc następujące wnioski:

* $bitmap[j]=1$, gdy $j \ll log_2n$

* $bitmap[j] \in \{0,1\}$ gdy $j \approx log_2n$

* $bitmap[j]=0$ gdy $j \gg log_2n$

Niech $R$ oznacza pozycję pierwszego zera licząc od lewej.

Algorytm Flajoleta-Martina mówi nam, że:

$$n\approx\frac{2^R}{\phi}, \phi=0.77351\dots$$

**Uwaga:**
W oryginalnej pracy opisującej algorytm pojawiają się dwie równości, których dowody są zbyt długie w stosunku do pojemności tego referatu

$$\mathbb{E}(R)=log_2(\phi n)$$

$$\sigma(R)=1.12\dots$$

Z powyższych równości wynika, że szacowanie liczby unikalnych elementów multizbioru może różnić się od rzeczywistej wartości o rzędy wielkości. W celu zniwelowania tego efektu można wykonać powyższy eksperyment $k$ razy, za każdym razem korzystając z innej funkcji hashującej i policzyć średnią z oszacowanych wartości. Wadą tego rozwiązania jest niska odporność na wartości odstające, które bez wątpienia będą się pojawiać - oszacowania różnią się przecież o całe rzędy wielkości. Innym sposobem, jest wybranie mediany ze zwróconych wartości. W tym wypadku wadą będzie ograniczony zakres możliwych wartości, ograniczony do przeskalowanych potęg dwójki. W praktyce wykonuje się $k\cdot l$ eksperymentów, z różną funkcją hashującą w każdym. Wtedy dla każdej wartości $i=1,\dots,k$ wykonujemy $l$ eksperymentów, z których wyciągamy medianę wyników, a następnie liczymy średnią median. W ten sposób jesteśmy w stanie znacząco poprawić oszacowanie, kosztem zwiększonej złożoności obliczeniowej.

### Implementacja algorytmu Flajoleta-Martina

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from HLL import HyperLogLog
from functools import reduce

In [3]:
def bit(y, k):
    return bin(y)[-(k+1)]
def rho(y):
    if y==0:
        return 0
    for i, b in enumerate(bin(y)[:1:-1]):
        if b == '1':
            return i
def find_R(bitmap):
    for i, b in enumerate(bitmap):
        if b == '0':
            return i
    return len(bitmap)

In [4]:
unique_words = {f'word{i}' for i in range(256000)}
unique_words_subsets = {k:{f'word{i}' for i in range(k*16000,(k+1)*16000)} for k in range(16)}
multisets = {k: np.random.choice(list(unique_words.difference(unique_words_subsets[k])), size=1500000) for k in range(16)}

In [6]:
M = multisets[0]
L = int(np.log2(len(M)))+10
phi=0.77351
k = 5
l = 5
medians=[]
for a, seed1 in enumerate(np.random.randint(0, 256, k)):
    results=[]
    for b, seed2 in enumerate(np.random.randint(0, 256, l)):
        bitmap=['0']*L
        for record in M:
            index = rho(abs(hash(f'{record}{seed1}{seed2}'))%2**L)
            if bitmap[index]=='0':
                bitmap[index]='1'
        str_bitmap = reduce(lambda x, y: x+y, bitmap)
        print(f'{str_bitmap}\n', end='\r', flush=True)
        results.append(2**find_R(str_bitmap)/phi)
        print(f'{a*k+b+1}/{k*l}', end='\r', flush=True)
    medians.append(np.median(results))
print(np.mean(medians))

111111111111111111000000000000
111111111111111110100000000000
111111111111111001000000000000
111111111111111111100000000000
111111111111111111010000000000
111111111111111110110000000000
111111111111111101010000000000
111111111111111111100000000000
111111111111111111100000000000
111111111111111001000000000000
111111111111111100100000000000
111111111111111111010000100000
111111111111111111000000000000
111111111111111101000000000000
111111111111111111110000000000
111111111111111110100000000000
111111111111111100010000000000
111111111111111101100100000000
111111111111111111010000000000
111111111111111110100000000000
111111111111111101001000000000
111111111111111111000000000000
111111111111111110100000000000
111111111111111111001000000000
111111111111111110000000000000
237231.32215485256


## HyperLogLog

Pochodzący z 2007 roku algorytm, współautorstwa Flajoleta, wprowadza następujące usprawnienia:

1. Zwiększona precyzja oszacowania liczby unikalnych elementów, przy złożoności obliczeniowej $O(|M|)$ - brak potrzeby wielokrotnego powtarzania eksperymentów
2. Stworzenie struktury danych HLL będącej reprezentacją zbioru unikalnych elementów multizbioru $M$, która udostępnia metody
    1. add
    2. count
    3. merge
    
#  
#  
### Idea algorytmu
W uwadze do algorytmu Flajoleta-Martina powiedzieliśmy, że wysoka wariancja $R$ wpływa na dużą niedokładność oszacowania liczby unikalnych elementów multizbioru $M$. W celu pozbycia się tego problemu, wykonywaliśmy wiele eksperymentów i liczyliśmy pewien agregat ich wyników. Algorytm HyperLogLog działa według podobnej idei. Zamiast wykonywać $m$ eksperymentów, tworzymy rejestry $B_i, i=0,\dots,m-1$ przechowujące najwyższą wartość $\rho$ z algorytmu Flajoleta-Martina dla danej grupy elementów. Niech $m$ będzie postaci $m=2^k$.

$hash(x)$, $y$, $\rho(y)$ oznaczają to samo co w algorytmie Flajoleta-Martina

Inicjalizujemy pustą strukturę HLL o $m$ rejestrach (każdy rejestr jest wypełniony zerami). Dla każdego rekordu z multizbioru $M$ obliczamy wartość jego hasha. Bity od $0$ do $k-1$ kodują indeks $j$ rejestru, w którym zapiszemy wystąpienie danego rekordu. Na podstawie pozostałych bitów liczymy $\rho$ i wstawiamy w dane miejsce jedynkę. Inaczej tę procedurę możemy zapisać jako:
***
$$j = y\ mod\ m$$
***
$$w = y - j$$
***
$$B_j=max\{B_j,\rho(w)\}$$

Liczność zbioru unikalnych elementów szacowana jest za pomocą średniej harmonicznej. Wiemy, że w każdym z rejestrów powinno być zapisanych około $\frac{n}{m}$ elementów.
$$ Z = \frac{1}{\sum_{i=0}^{m-1}2^{-B_i}}$$

$$\frac{n}{m} \approx mZ$$

Biorąc pod uwagę konflikty hashowania, możemy wyliczyć stałą szacowania $\alpha_m$. Wtedy

$$n = \alpha_m m^2 Z$$

$$\alpha_m = \frac{1}{m \int_0^\infty(log_2(\frac{2+u}{1+u}))^m du}$$

Obliczenie $\alpha_m$ nie jest najprostszą czynnością, dlatego wykorzystuje się poniższe oszacowanie:

$$\alpha_m = 
\begin{cases}
0.673, m=16\\
0.697, m=32\\
0.709, m=64\\
\frac{0.7213}{1+\frac{1.079}{m}}, m \geq 128
\end{cases}$$

## Merge

Niech $hll_1, hll_2$ będą strukturami HLL, obie mają m rejestrów, a $j$-ty rejestr strutury $hll_i$ oznaczmy jako $hll_{i,j}$. Wtedy

$$union(hll_1, hll_2)[j] = max\{hll_{1,j},hll_{2,j}\}$$

In [33]:
b = 16
hll1 = HyperLogLog(b)
hll2 = HyperLogLog(b)

In [34]:
M0 = multisets[0]
M1 = multisets[1]

In [35]:
for record in M0:
    hll1.add(record)
    
for record in M1:
    hll2.add(record)

In [42]:
len(set(M0).union(set(M1)))

255941

In [36]:
np.unique(M0).size

239534

In [37]:
np.unique(M1).size

239571

In [38]:
hll1.cardinality()

240681.13362848014

In [39]:
hll2.cardinality()

240529.29328457054

In [40]:
hll1.merge(hll2)
hll1.cardinality()

256336.23671754944

In [ ]:
intersection = hll1.cardinality() + hll2.cardinality() - 